# TRABAJO REDES NEURONALES Y APRENDIZAJE ESTADÍSTICO


Autores: Miguel Bande Rodríguez y Octavian Rotita Ion

# Importamos las librerías necesarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gpd
import folium
from ipywidgets import interact, IntSlider
from IPython.display import display
import seaborn as sns
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Importamos la base de datos

In [ ]:
# La base de datos se encuentra en formato CSV

# Cargamos la base de datos
df = pd.read_csv('./fires-all.csv')

# 1. Preprocesado de la base de datos

## 1.1 Visualización de la base de datos

In [ ]:
# Mostramos las primeras 5 filas
print(df.head())

In [ ]:
df.info()

Veamos lo que significa cada una de las variables:

| Nombre del campo   | Descripción |
|--------------------|-------------|
| id                | Identificador del incendio |
| superficie        | Superficie forestal quemada en hectáreas |
| fecha            | Fecha de detección del incendio (formato yyyy-mm-dd) |
| lat              | Latitud geográfica del origen del incendio |
| lng              | Longitud geográfica del origen del incendio |
| latlng_explicit  | Indica si se dispone de coordenadas geográficas del incendio (1) o se han usado las coordenadas del municipio de origen del incendio (0) |
| comunidad        | Identificador de la CC.AA. |
| provincia        | Identificador de la provincia |
| municipio        | Nombre del municipio |
| causa           | Identificador de la causa del incendio |
| causa_supuesta  | Es ‘1’ si la causa es supuesta, si no en blanco |
| causa_desc      | Identificador de la descripción de la causa del incendio |
| muertos         | Número de muertos en el incendio |
| heridos         | Número de heridos en el incendio |
| time_ctrl       | Tiempo transcurrido hasta entrar en fase de control del incendio (en minutos) |
| time_ext        | Tiempo transcurrido hasta la extinción del incendio (en minutos) |
| personal        | Número de personas que han participado en la extinción del incendio (incluye técnicos, agentes forestales, brigadas, bomberos, voluntarios, guardias civiles y ejército) |
| medios          | Número de medios terrestres y aéreos que han participado en la extinción del incendio (incluye autobombas, bulldozers, tractores, aviones y otros) |
| gastos          | Gastos de extinción asociados al incendio tal y como figuran en EGIF |
| perdidas        | Pérdidas económicas asociadas al incendio tal y como figuran en EGIF |


Haciendo una breve exploración de la base de datos, podemos ver que no tenemos información de latitud y longitud hasta el año 1983. Además, a partir de 2017 hay provincias que han dejado de proporcionar información.

Vamos a convertir a datetime

In [ ]:
# Convirtamos la casilla fecha a datetime

df['fecha'] = pd.to_datetime(df['fecha'])

In [ ]:
# Hemos decidido eliminar las inconsistencias relacionadas con la fecha que hemos descrito anteriormente. Es por esto que nos quedaremos con los datos a partir del 1 de enero del 1983 hasta el 31 de diciembre de 2016

df = df[(df['fecha'] >= '1983-01-01') & (df['fecha'] <= '2016-12-31')]
df.info()


Podemos ver que seguimos teniendo valores missing en latitud y longitud, visualicémolos y decidamos que hacer con ellos.

In [ ]:
# Veamos los missings de latitud y longitud

df[df['lat'].isnull()]

Vemos que dichos valores missing en latitud y en longitud se corresponden con categorías en las cuales municipio se corresponde con: otra provincia, portugal o francia. Almodovar de Monterrey es un municipio que ha pasado por diversos procesos de separación y agregación (en cuanto a terrenos). Como es solamente uno el que no tiene valores en latitud y longitud, eliminémoslo.

In [ ]:
df.drop(df[df['lat'].isnull()].index, inplace=True)

df.info()

Como la información del municipio la podemos obtener en función de las variables latitud y longitud, vamos a eliminar dicha variable del dataset.

In [ ]:
# Eliminemos la variable municipio

df.drop('municipio', axis=1, inplace=True)
df.drop('idmunicipio', axis=1, inplace=True)



In [ ]:
df.info()

Estudiemos ahora las comunidades y las provincias

In [ ]:
# En primer lugar veamos la codificación de las comunidades autónomas (estas se encuentran en la base de datos del ministerio de transición ecológica (EGIF))

dict_ccaa = {
    '1': 'Euskadi',
    '2': 'Cataluña',
    '3': 'Galicia',
    '4': 'Andalucía',
    '5': 'Asturias',
    '6': 'Cantabria',
    '7': 'La Rioja',
    '8': 'Murcia',
    '9': 'C. Valenciana',
    '10': 'Aragón',
    '11': 'Castilla la Mancha',
    '12': 'Canarias',
    '13': 'Navarra',
    '14': 'Extremadura',
    '15': 'Illes Balears',
    '16': 'Madrid',
    '17': 'Castilla y León',
    '18': 'Ceuta',
    '19': 'Melilla',
    '99': 'Otro país'
}

In [ ]:
# Hagamos un value counts de idcomunidad para ver si los datos son consistentes con el mapa. Juntémolos con el diccionario para saber que comunidad autónoma es cada id

df['idcomunidad'] = df['idcomunidad'].astype(str)
df['comunidad'] = df['idcomunidad'].map(dict_ccaa)

df['comunidad'].value_counts()

Vemos que no aparece 'Otro país', por lo que el filtrado anterior lo hemos hecho correctamente.

In [ ]:
# Hacemos ahora un diccionario para representar las causas de los incendios

causas_incendios = {
    1: "Fuego por rayo",
    2: "Fuego por accidente o negligencia",
    3: "Fuego por accidente o negligencia",
    4: "Fuego intencionado",
    5: "Fuego por causa desconocida",
    6: "Incendio reproducido"
}

### Distribución geográfica de los incendios

In [ ]:
# Extraer el año de la fecha
df["año"] = df["fecha"].dt.year
colores = ['yellow', 'blue', 'blue', 'red', 'purple', 'orange']

# Obtener rango de años disponibles
min_año, max_año = df["año"].min(), df["año"].max()

def mostrar_mapa(año):
    # Filtrar los datos por el año seleccionado
    df_filtrado = df[df["año"] == año]

    # Crear un mapa centrado en España
    mapa = folium.Map(location=[40.0, -3.5], zoom_start=5)

    # Agregar los puntos al mapa
    for _, row in df_filtrado.iterrows():
        folium.CircleMarker(
            location=[row["lat"], row["lng"]],
            radius=(row["superficie"]-20)/215,
            color=colores[row["causa"]-1],
            fill=True,
            fill_color= colores[row["causa"]-1],
            fill_opacity=0.5,
        ).add_to(mapa)

    return mapa

# Crear el widget de selección de año
interact(mostrar_mapa, año=IntSlider(min=min_año, max=max_año, step=1, value=min_año))


In [ ]:
# Filtrar los datos por el año seleccionado
df_filtrado = df[df["año"] == 1983]

# Crear un mapa centrado en España
mapa = folium.Map(location=[40.0, -3.5], zoom_start=5)

# Agregar los puntos al mapa
for _, row in df_filtrado.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lng"]],
        radius=(row["superficie"]-20)/215,
        color=colores[row["causa"]-1],
        fill=True,
        fill_color= colores[row["causa"]-1],
        fill_opacity=0.5,
    ).add_to(mapa)

mapa.save("mapa_interactivo.html")


Visualización de algunos histogramas

### Número de incendios por año

In [ ]:
incendios_por_año = df["año"].value_counts().reset_index()
incendios_por_año.columns = ["Año", "Cantidad de Incendios"]
incendios_por_año = incendios_por_año.sort_values("Año")

fig = px.bar(
    incendios_por_año, 
    x="Año", 
    y="Cantidad de Incendios", 
    title="Número de Incendios en España por Año",
    labels={"Año": "Año", "Cantidad de Incendios": "Número de Incendios"},
    text_auto=True,
)

# Mostrar el gráfico
fig.show()


### Distribución de superficie quemada

In [ ]:
fig = px.histogram(
    df, x="superficie", nbins=40, 
    title="Distribución de Superficie Quemada",
    labels={"superficie": "Superficie quemada (ha)"},
)
fig.show()


### Distribución de incendios por mes y año

In [ ]:
import plotly.graph_objects as go

# Agrupar por año y mes
df["mes"] = df["fecha"].dt.month
heatmap_data = df.groupby(["año", "mes"]).size().reset_index(name="incendios")

fig = px.density_heatmap(
    heatmap_data, x="mes", y="año", z="incendios",
    title="Incendios por Mes y Año",
    labels={"mes": "Mes", "año": "Año", "incendios": "Cantidad"},
    color_continuous_scale="reds"
)
fig.show()


### Relación entre tiempo de extinción y superficie quemada

In [ ]:
fig = px.scatter(
    df, y="superficie", x = "time_ext",
    title="Relación entre Tiempo de Extinción y Superficie Quemada",
    labels={"time_ext": "Tiempo de Extinción (min)", "superficie": "Superficie Quemada (ha)"},
    log_y=True,
    log_x=True
)
fig.show()


In [ ]:
fig = px.scatter(
    df, y="superficie", x = "time_ctrl",
    title="Relación entre Tiempo de Control y Superficie Quemada",
    labels={"time_ext": "Tiempo de Control (min)", "superficie": "Superficie Quemada (ha)"},
    log_y=True,
    log_x=True
)
fig.show()


In [ ]:
# Mapear las causas al nombre correspondiente
df["causa_map"] = df["causa"].map(causas_incendios)

# Contar los incendios por causa
causas_contadas = df["causa_map"].value_counts().reset_index()
causas_contadas.columns = ["Causa", "Cantidad"]

# Crear gráfico de pastel
fig = px.pie(
    causas_contadas, 
    names="Causa", 
    values="Cantidad",
    title="Distribución de Causas de Incendios",
)

fig.show()


### Estadísticos de las variables presentes en el dataset

In [ ]:
df.describe().transpose()

In [ ]:
# Vemos que hay varios casos con pérdidas negativas. Esto no tiene sentido alguno. Vamos a eliminar estas inconsistencias.

df = df[df['perdidas'] >= 0]

### Tiempo promedio de extinción de incendios por comunidad

In [ ]:
# Asegurar que la columna "comunidad" es categórica y "time_ext" es numérico
df["comunidad"] = df["comunidad"].astype(str)
df["time_ext"] = pd.to_numeric(df["time_ext"], errors="coerce")

# Calcular el tiempo medio de extinción por comunidad
extincion_por_comunidad = df.groupby("comunidad")["time_ext"].mean().reset_index()

# Crear el gráfico de barras
fig = px.bar(
    extincion_por_comunidad,
    x="comunidad",
    y="time_ext",
    title="Tiempo Promedio de Extinción de Incendios por Comunidad",
    labels={"comunidad": "Comunidad Autónoma", "time_ext": "Tiempo de Extinción (min)"},
    text_auto=True,
)

# Rotar etiquetas del eje X para mejor visibilidad
fig.update_layout(xaxis_tickangle=-45)

# Mostrar el gráfico
fig.show()

### Superficie media quemada por comunidad

In [ ]:
# Asegurar que la columna "comunidad" es categórica y "time_ext" es numérico
df["comunidad"] = df["comunidad"].astype(str)

# Calcular la superficie media quemada por comunidad
extincion_por_comunidad = df.groupby("comunidad")["superficie"].mean().reset_index()

# Crear el gráfico de barras
fig = px.bar(
    extincion_por_comunidad,
    x="comunidad",
    y="superficie",
    title="Superficie media quemada por Comunidad",
    labels={"comunidad": "Comunidad Autónoma", "time_ext": "Tiempo de Extinción (min)"},
    text_auto=True,
)

# Rotar etiquetas del eje X para mejor visibilidad
fig.update_layout(xaxis_tickangle=-45)

# Mostrar el gráfico
fig.show()

In [ ]:
# Especificar el orden de las comunidades
orden_comunidades = [
    "C. Valenciana", "Aragón", "Cantabria", "Cataluña", "Asturias", "La Rioja", 
    "Euskadi", "Andalucía", "Galicia", "Castilla y León", "Castilla la Mancha", 
    "Canarias", "Illes Balears", "Extremadura", "Murcia", "Madrid", "Navarra", "Ceuta"
]

# Crear el gráfico de barras
plt.figure(figsize=(20, 5))
sns.barplot(
    data=extincion_por_comunidad,
    x="comunidad",
    y="superficie",
    order=orden_comunidades
)

# Añadir título y etiquetas
plt.xlabel("Comunidad Autónoma")
plt.ylabel("Superficie Media Quemada")

# Rotar etiquetas del eje X para mejor visibilidad
plt.xticks(rotation=-45)

# Mostrar el gráfico
plt.show()


### Superficie total quemada por Comunidad

In [ ]:
# Asegurar que la columna "comunidad" es categórica y "time_ext" es numérico
df["comunidad"] = df["comunidad"].astype(str)

# Calcular la superficie total quemada por comunidad
extincion_por_comunidad = df.groupby("comunidad")["superficie"].sum().reset_index()

# Crear el gráfico de barras
fig = px.bar(
    extincion_por_comunidad,
    x="comunidad",
    y="superficie",
    title="Superficie total quemada por Comunidad",
    labels={"comunidad": "Comunidad Autónoma", "time_ext": "Tiempo de Extinción (min)"},
    text_auto=True,
)

# Rotar etiquetas del eje X para mejor visibilidad
fig.update_layout(xaxis_tickangle=-45)

# Mostrar el gráfico
fig.show()

In [ ]:
# Lista de variables a analizar con boxplots
variables_boxplot = ["superficie", "time_ext", "time_ctrl", "muertos", "heridos", "personal", "gastos", "perdidas"]

# Crear gráficos de boxplot para cada variable
fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(20, 4))
fig.suptitle("Detección de Outliers en Variables Clave", fontsize=16)

# Dibujar cada boxplot
for ax, variable in zip(axes.flatten(), variables_boxplot):
    sns.boxplot(y=df[variable], ax=ax)

# Ajustar el layout para mejor visualización
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# Lista de columnas a transformar
cols_to_transform = ["superficie", "time_ext", "time_ctrl", "muertos", "heridos", "personal", "gastos", "perdidas"]


# Aplicar la transformación log(1 + x)
df[cols_to_transform] = df[cols_to_transform].apply(lambda x: np.log1p(x))

In [ ]:
df.info()

### Representación box-plot para detección de outliers

In [ ]:
# Lista de variables a analizar con boxplots
variables_boxplot = ["superficie", "time_ext", "time_ctrl", "muertos", "heridos", "personal", "gastos", "perdidas"]

# Crear gráficos de boxplot para cada variable
fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(20, 4))
fig.suptitle("Detección de Outliers en Variables Clave", fontsize=16)

# Dibujar cada boxplot
for ax, variable in zip(axes.flatten(), variables_boxplot):
    sns.boxplot(y=df[variable], ax=ax)

# Ajustar el layout para mejor visualización
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
df.info()

In [ ]:
# Veamos los diagramas de caja y bigote de superficie (logaritmo) por causa

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x="causa_map", y="superficie")
plt.tight_layout()


In [ ]:
# Veamos los diagramas de caja y bigote de superficie (logaritmo) por causa

plt.figure(figsize=(21, 9))
sns.boxplot(data=df, x="comunidad", y="superficie")
plt.tight_layout()


## 1.2. Selección de variables predictoras

### Matriz de correlaciones

In [ ]:
# Seleccionar solo columnas de interés para el análisis de correlación
variables_corr = ["superficie", "time_ext", "time_ctrl", "personal", "gastos", "perdidas"]
df_numeric = df[variables_corr].apply(pd.to_numeric, errors="coerce")  # Convertir a numérico y forzar errores a NaN

# Calcular la matriz de correlación
correlation_matrix = df_numeric.corr()

# Crear el heatmap de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación entre Variables de Incendios")
plt.show()


Observamos que las variables más correlacionadas entre sí son el tiempo de control y el tiempo de extinción, pero no están altamente correlacionadas, por lo que decidimos usar ambas variables para entrenamiento de modelo. Además, conocer el tiempo de extinción y el tiempo de contro simultáneamente nos puede indicar la magnitud de superficie quemada, ya que un incendio que ha tardado más tiempo en extinguirse desde el momento que se considera controlado ha podido ser más devastador.

### Importancia de variables con modelos basados en árboles

In [ ]:
# Definir variables predictoras y objetivo
X = df[["time_ext", "time_ctrl", "personal", "gastos", "perdidas"]]
y = df["superficie"]

# Entrenar modelo Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Extraer importancia de variables
importances = pd.DataFrame({"Variable": X.columns, "Importancia": rf.feature_importances_})
importances = importances.sort_values(by="Importancia", ascending=False)

In [ ]:
plt.bar(importances["Variable"], importances["Importancia"], color="red", alpha = 0.5, edgecolor="black")
plt.title("Importancia de Variables en el Modelo")
plt.xlabel("Variable")
plt.ylabel("Importancia")
plt.show()


### Utilizar pruebas estadísticas para seleccionar las mejores características con K Best

In [ ]:
# Aplicar prueba estadística
selector = SelectKBest(score_func=f_regression, k=3)
X_new = selector.fit_transform(X, y)

# Ver qué variables fueron seleccionadas
selected_features = X.columns[selector.get_support()]
selected_features


# 2. Entrenamiento red neuronal

## 2.1 División del conjunto en entrenamiento y test (con el fin de evaluar nuestro regresor)

In [ ]:
# Para ello, vamos a usar la librería de sklearn train_test_split.

# En primer lugar nos quedamos con las variables predictoras y la variable objetivo

X = df[['time_ext', 'time_ctrl', 'personal', 'perdidas', 'idcomunidad']]
# X = df[['idcomunidad','causa','time_ext', 'time_ctrl', 'personal', 'gastos', 'perdidas']]
y = df['superficie']

# Dividimos los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 2.2 Codificación de las variables categóricas y normalización de las variables numéricas


In [ ]:
# Variables categóricas y numéricas
categorical_features = ['idcomunidad']#['causa']#, 'idcomunidad']
numerical_features = ['time_ext', 'time_ctrl', 'personal', 'perdidas']

# Crear pipeline con One-Hot Encoding y Normalización
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_features),  # Normalizar variables numéricas
    ('cat', OneHotEncoder(drop='first'), categorical_features)  # One-Hot Encoding en categóricas
])

# Transformar los datos
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [ ]:
X_train_transformed.shape, X_test_transformed.shape

## 2.3 Uso de cross-validation para calcular los hiperparámetros óptimos de la red neuronal

In [ ]:
# Definir la red neuronal base
mlp = MLPRegressor(max_iter=500, random_state=42)

# Definir los hiperparámetros a optimizar
param_grid = {
    'hidden_layer_sizes': [(24,), (32,), (32, 16)],  # Diferentes arquitecturas
    'activation': ['relu', 'tanh'],  # Funciones de activación
    'solver': ['adam'],  # Algoritmos de optimización
    'alpha': [0.0001, 0.001, 0.01],  # Regularización L2 (distribución uniforme)
    'learning_rate': ['constant', 'adaptive'],  # Tipo de tasa de aprendizaje
}


# Ejecutar la búsqueda con Cross-Validation
grid_search = GridSearchCV(mlp, 
                            param_grid,
                             cv=5, # Cross-validation con 5 folds
                             scoring='neg_mean_squared_error',  # Minimizar MAE
                             n_jobs=-1, # Usar todos los núcleos disponibles
                             verbose=2) # Mostrar progreso

grid_search.fit(X_train_transformed, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)

# Evaluar el mejor modelo
best_mlp = grid_search.best_estimator_
y_pred = best_mlp.predict(X_test_transformed)

# Evaluación
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📉 MAE: {mae:.2f} hectáreas")
print(f"📉 MSE: {mse:.2f} hectáreas²")
print(f"📊 R² Score: {r2:.2f}")



In [ ]:
# Vamos a entrenar una red neuronal sola

# Crear la red neuronal
mlp = MLPRegressor(hidden_layer_sizes=(32,16),  # 2 capas ocultas con 64 y 32 neuronas
                    activation='tanh',  # Función de activación ReLU
                    solver='adam',  # Optimizador Adam
                    alpha=0.0001,  # Regularización L2
                    learning_rate='constant',  # Ajuste dinámico de tasa de aprendizaje
                    max_iter=500,  # Número de iteraciones de entrenamiento
                    random_state=42)

# Entrenar la red
mlp.fit(X_train_transformed, y_train)

# Predicciones en datos de prueba
y_pred = mlp.predict(X_test_transformed)

In [ ]:
# 📊 Evaluación del modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📉 MAE (Error Absoluto Medio): {mae:.2f} hectáreas")
print(f"📉 MSE (Error Cuadrático Medio): {mse:.2f} hectáreas²")
print(f"📊 R² Score (Precisión del modelo): {r2:.2f}")


### Veamos las predicciones

In [ ]:
import folium
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement


color = ['yellow', 'blue', 'blue', 'red', 'purple', 'orange']
# Definir umbral de error (en hectáreas)
umbral_error = 0  # Modificar según necesidad

# Crear un DataFrame con los resultados
resultados = pd.DataFrame({
    'Real': y_test,
    'Predicho': y_pred
})

# Calcular el error absoluto
resultados['Error'] = np.abs(resultados['Real'] - resultados['Predicho'])

resultados['id'] = X_test.index  # Asigna el índice de X_test como identificador
df['id'] = df.index  # Si df tampoco tiene id, usa su índice

# 📌 Asegurar que df tiene 'id', 'lat', y 'lng' y hacer la unión correctamente
resultados = resultados.merge(df, on='id', how='left')  # Cambia 'id' si el nombre es distinto

# Filtrar puntos con error superior al umbral
errores_significativos = resultados[resultados['Error'] > umbral_error]

# Solución: Eliminar NaN antes de usar Folium
errores_significativos = errores_significativos.dropna(subset=['lat', 'lng'])

# Crear mapa centrado en la media de los puntos
mapa = folium.Map(location=[errores_significativos['lat'].mean(), 
                            errores_significativos['lng'].mean()], zoom_start=6)


# Agrupar los marcadores
marker_cluster = MarkerCluster().add_to(mapa)


# Agregar puntos al mapa
for _, fila in errores_significativos.iterrows():
    folium.CircleMarker(
        location=[fila['lat'], fila['lng']],
        radius=fila['superficie'],
        color=color[fila["causa"]-1],
        fill=True,
        fill_color=color[fila["causa"]-1],
        fill_opacity=0.6,
        popup=f"Real: {np.exp(fila['Real'])-1:.2f} ha \n Predicho: {np.exp(fila['Predicho'])-1:.2f} ha \n Error: {fila['Error']:.2f} ha \n Causa: {fila['causa_map']}"
    ).add_to(marker_cluster)


# Mostrar el mapa interactivo
mapa

In [ ]:
mapa.save("mapa_interactivo_predicciones.html")


### Mapa interactivo de los incendios cuya predicción difiere 19 hectáreas del valor real

In [ ]:
import folium
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement


color = ['yellow', 'blue', 'blue', 'red', 'purple', 'orange']
# Definir umbral de error (en hectáreas)
umbral_error = 3  # Modificar según necesidad

# Crear un DataFrame con los resultados
resultados = pd.DataFrame({
    'Real': y_test,
    'Predicho': y_pred
})

# Calcular el error absoluto
resultados['Error'] = np.abs(resultados['Real'] - resultados['Predicho'])

resultados['id'] = X_test.index  # Asigna el índice de X_test como identificador
df['id'] = df.index  # Si df tampoco tiene id, usa su índice

# 📌 Asegurar que df tiene 'id', 'lat', y 'lng' y hacer la unión correctamente
resultados = resultados.merge(df, on='id', how='left')  # Cambia 'id' si el nombre es distinto

# Filtrar puntos con error superior al umbral
errores_significativos = resultados[resultados['Error'] > umbral_error]

# Solución: Eliminar NaN antes de usar Folium
errores_significativos = errores_significativos.dropna(subset=['lat', 'lng'])

# Crear mapa centrado en la media de los puntos
mapa = folium.Map(location=[errores_significativos['lat'].mean(), 
                            errores_significativos['lng'].mean()], zoom_start=6)


# Agrupar los marcadores
marker_cluster = MarkerCluster().add_to(mapa)


# Agregar puntos al mapa
for _, fila in errores_significativos.iterrows():
    folium.CircleMarker(
        location=[fila['lat'], fila['lng']],
        radius=fila['superficie'],
        color=color[fila["causa"]-1],
        fill=True,
        fill_color=color[fila["causa"]-1],
        fill_opacity=0.6,
        popup=f"Real: {np.exp(fila['Real'])-1:.2f} ha \n Predicho: {np.exp(fila['Predicho'])-1:.2f} ha \n Error: {fila['Error']:.2f} ha \n Causa: {fila['causa_map']}"
    ).add_to(marker_cluster)


# Mostrar el mapa interactivo
mapa

In [ ]:
mapa.save("mapa_interactivo_predicciones_3.html")